In [ ]:
# Modules
import tweepy, json
import pandas as pd
import matplotlib.pyplot as plt
import requests
from PIL import Image
from io import BytesIO

# Athentication
consumer_key = '#'
consumer_secret = '#'

access_token = '#'
access_token_secret = '#'

print('Connecting to Twitter API....')
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
# Saving images in the retrieved tweets
def download(search_results, location='data'):
    for i, tweet in enumerate(search_results):
        for j,media in enumerate(tweet.entities.get("media",[{}])):
            #checks if there is any media-entity
            if media.get("type",None) == "photo":
                # checks if the entity is of the type "photo"
                image_content=requests.get(media["media_url"])
                # save images
                img = Image.open(BytesIO(image_content.content))
                img.save(f"{location}/{str(tweet.id)}_{str(j)}.png")

# Scrapping images at `#faceapp`
original code was provided [here](https://bhaskarvk.github.io/2015/01/how-to-use-twitters-search-rest-api-most-effectively./)
- Added the ability to download images for each tweet
- Specified the `lang` to be English

### Scraps all images from the hashtag
- Real images
- Fake generated images
- A collage of Real and Fake images
- Non-relevant images

In [ ]:
searchQuery = '#faceapp'  # this is what we're searching for
maxTweets = 10000000 # Some arbitrary large number
tweetsPerQry = 100  # this is the max the API permits
tweetCount = 0
max_id = -1
sinceId = None

while tweetCount < maxTweets:
    try:
        if (max_id <= 0):
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, lang='en', count=tweetsPerQry)
            else:
                new_tweets = api.search(q=searchQuery, lang='en', count=tweetsPerQry,
                                        since_id=sinceId)
        else:
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, lang='en', count=tweetsPerQry,
                                        max_id=str(max_id - 1))
            else:
                new_tweets = api.search(q=searchQuery, lang='en', count=tweetsPerQry,
                                        max_id=str(max_id - 1),
                                        since_id=sinceId)
        if not new_tweets:
            print("No more tweets found")
        #for tweet in new_tweets:
        download(new_tweets)
            
        tweetCount += len(new_tweets)
        print("Downloaded {0} tweets".format(tweetCount))
        max_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        # Just exit if any error
        print("some error : " + str(e))
        break

# Downloading more real images to balance the data 
- This time we are using `#new_avatar` as people will tend to put their real images
- All the images are downloaded to the `more_original` folder

In [ ]:
searchQuery = '#new_avatar'  # this is what we're searching for
maxTweets = 10000 # Some arbitrary large number
tweetsPerQry = 100  # this is the max the API permits
tweetCount = 0
max_id = -1
sinceId = None


while tweetCount < maxTweets:
    try:
        if (max_id <= 0):
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry)
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        since_id=sinceId)
        else:
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        max_id=str(max_id - 1))
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        max_id=str(max_id - 1),
                                        since_id=sinceId)
        if not new_tweets:
            print("No more tweets found")
            break
        #for tweet in new_tweets:
        download(new_tweets, location='more_original')
            
        tweetCount += len(new_tweets)
        print("Downloaded {0} tweets".format(tweetCount))
        max_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        # Just exit if any error
        print("some error : " + str(e))
        break